In [5]:
import os
from pathlib import Path

In [6]:
%pwd

'd:\\Projects\\ViT_from_scratch\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'd:\\Projects\\ViT_from_scratch'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir : Path
    unzip_dir : Path

In [10]:
from src.vision_Transformer.constants import *
from src.vision_Transformer.utils.common import create_directories , read_yaml

In [11]:
class ConfigurationManager:
    def __init__(self , config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config =read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            unzip_dir = config.unzip_dir,
        )

        return data_ingestion_config

In [22]:
import torchvision.datasets as datasets
from src.vision_Transformer.logging import logger

In [23]:
import  torchvision.transforms as transforms
pre_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.5) , std=  (0.5)) 
    #  helps to make model converge faster 
    # 2. helps to make numerical computations stable
])

In [44]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig , pre_transform):
        self.config = config

        print(self.config.root_dir)
        self.pre_transform = pre_transform 


    def download_dataset(self):
        try:
          train_dataset = datasets.CIFAR10(
             root = self.config.root_dir,
             download=True,
             train = True,
             transform= self.pre_transform
          )
          logger.info(f"Train Dataset downloaded at {self.config.root_dir}")

          dataset = datasets.CIFAR10(
             root = self.config.root_dir,
             download=True,
             train = False,
             transform= self.pre_transform
          )
          logger.info(f"Test Dataset downloaded at {self.config.root_dir}")

        except Exception as e:
          print(f'An exception {e} occurred')

In [45]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(data_ingestion_config , pre_transform)
    data_ingestion.download_dataset()

except Exception as e:
    print(f'An exception {e} occurred')

[2025-07-31 22:21:35,403 : INFO : common  : yaml file config\config.yaml was read succesfully]
[2025-07-31 22:21:35,411 : INFO : common  : yaml file params.yaml was read succesfully]
[2025-07-31 22:21:35,414 : INFO : common  : Created directory at : artifacts]
[2025-07-31 22:21:35,415 : INFO : common  : Created directory at : artifacts/data_ingestion]
artifacts/data_ingestion
[2025-07-31 22:21:36,455 : INFO : 2639109126  : Train Dataset downloaded at artifacts/data_ingestion]
[2025-07-31 22:21:37,196 : INFO : 2639109126  : Test Dataset downloaded at artifacts/data_ingestion]
